In [154]:
#### LIBRAIRIES ####
import pandas as pd
import numpy as np

In [155]:
#### FUNCTIONS ####

# Describe

def describe_col(df, col):
    '''Display name, statistics, and distribution of values of a column.'''
    print(f'{upper_spaced(col)} ({col})')
    print(f'\nGeneral stats')
    print(df[col].describe())
    print(f'\nValue distribution')
    print(df[col].value_counts())
    print(f'\nMissing values')
    print(df[col].isna().sum())
    
def distrib_missing_values(df, cols):
    '''Display and return the count of missing values for the selected columns.'''
    d_missing_values = {'id': [], 'count_missing_values': []}
    for i in df.index:
        d_missing_values['id'].append(i)
        d_missing_values['count_missing_values'].append(df.loc[i, cols].isna().sum())
    df_missing_values = pd.DataFrame.from_dict(d_missing_values)
    print(df_missing_values['count_missing_values'].value_counts())
    return d_missing_values
    
def describe_unique_values(df, col):
    '''Display name and unique values of a column.'''
    print(f'{upper_spaced(col)} ({col})')
    print(f'\nUnique values')
    print(sorted(df[col].unique()))
    
def upper_spaced(my_str):
    '''Transform a CamelCase string into a CAMEL CASE upper spaced string.'''
    return re.sub(r"(\w)([A-Z])", r"\1 \2", my_str).upper()


# Homogeneise

def homogeneise_column_names(df, replace_dict):
    '''Homegeneise column names based on a replacement dictionary.'''
    for old, new in replace_dict.items():
        df.columns = df.columns.str.replace(old, new)

def homogeneise_values(df, replace_dict):
    '''Homegeneise values based on a replacement dictionary.'''
    for old, new in replace_dict.items():
        df.replace(old, new, inplace = True)

In [156]:
#### DATA IMPORT ####
original_data = pd.read_csv('../data/HR_Data.csv')
data = original_data.copy()
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21,1.0
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98,0.0
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15,0.0
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39,1.0
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72,0.0


In [157]:
#### DATA SHAPE ####
data.shape

(2129, 14)

In [158]:
#### DATA TYPES ####
data.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [159]:
#### DATA STATISTICAL DESCRIPTION ####
data.describe()

,enrollee_id,city_development_index,training_hours,target
count,2129.000000,2129.000000,2129.000000,2129.000000
mean,16861.614843,0.824984,64.983091,0.255049
std,9576.846029,0.125074,60.238660,0.435991
min,3.000000,0.448000,1.000000,0.000000
25%,8562.000000,0.698000,23.000000,0.000000
50%,16816.000000,0.903000,47.000000,0.000000
75%,25129.000000,0.920000,86.000000,1.000000
max,33353.000000,0.949000,334.000000,1.000000


In [160]:
#### UNIQUE VALUES CATEGORICAL COLUMNS ####

l_categorical_cols = ['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

for col in l_categorical_cols:
    describe_col(data, col)
    print('\n--------------------')

GENDER (gender)

General stats
count     1621
unique       3
top       Male
freq      1460
Name: gender, dtype: object

Value distribution
Male      1460
Female     137
Other       24
Name: gender, dtype: int64

Missing values
508

--------------------
RELEVENT_EXPERIENCE (relevent_experience)

General stats
count                        2129
unique                          2
top       Has relevent experience
freq                         1524
Name: relevent_experience, dtype: object

Value distribution
Has relevent experience    1524
No relevent experience      605
Name: relevent_experience, dtype: int64

Missing values
0

--------------------
ENROLLED_UNIVERSITY (enrolled_university)

General stats
count              2098
unique                3
top       no_enrollment
freq               1519
Name: enrolled_university, dtype: object

Value distribution
no_enrollment       1519
Full time course     435
Part time course     144
Name: enrolled_university, dtype: int64

Missing values
31



In [161]:
#### MISSING VALUES ####
data.isna().sum()

enrollee_id                 0
city                        0
city_development_index      0
gender                    508
relevent_experience         0
enrolled_university        31
education_level            52
major_discipline          312
experience                  5
company_size              622
company_type              634
last_new_job               40
training_hours              0
target                      0
dtype: int64

In [162]:
#### DISTRIBUTION OF ROWS WITH MISSING VALUES ####
count_missing_values = {id:data.loc[[id]].isna().sum().sum() for id in data.index}
dct_count_missing_values = {'id':list(count_missing_values.keys()), 'count': list(count_missing_values.values())}
df_count_missing_values = pd.DataFrame(dct_count_missing_values)
df_count_missing_values.sort_values(by='count', ascending = False)
df_count_missing_values['count'].value_counts()

0    1019
1     434
2     376
3     215
4      60
5      19
6       4
7       2
Name: count, dtype: int64

In [163]:
#### DROP ROWS WITH MORE THAN 2 MISSING VALUES ####
data.drop(data[df_count_missing_values['count'] > 2].index, axis = 0, inplace = True)

In [164]:
#### FILL NaN WITH 'Unknown' #####
data.fillna('Unknown', inplace = True)

In [165]:
#### TYPO CORRECTION ####
# Columns
replace_dict_colnames = {
                    ' ': '',
                    '-': '_',
                    'relevent': 'relevant'
                    }

homogeneise_column_names(data, replace_dict_colnames)

# Values
replace_dict_values = {
                        '<10': '0-10',
                        '10/49': '10-49',
                        'no_enrollment': 'No enrollment',
                        'Has relevent experience': 'Has relevant experience',
                        'No relevent experience': 'No relevant experience'
                        }

homogeneise_values(data, replace_dict_values)

In [166]:
#### CATEGORICAL COLUMNS ORDERING ####
cats_education_level = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd', 'Unknown']
data.education_level = pd.Categorical(data.education_level, ordered = True, categories = cats_education_level)

cats_enrolled_university = ['No enrollment', 'Part time course', 'Full time course', 'Unknown']
data.enrolled_university = pd.Categorical(data.enrolled_university, ordered=True, categories=cats_enrolled_university)

cats_company_size = ['0-10', '10-49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+', 'Unknown']
data.company_size = pd.Categorical(data.company_size, ordered=True, categories=cats_company_size)

# The order of below categories is artificial (e.g. no <> relationship between Male and Female)
# and based on cardinality
# but it enables us to put 'Other' and 'Unknown' at the very right

cats_gender = ['Male', 'Female', 'Other', 'Unknown']
data.gender = pd.Categorical(data.gender, ordered=True, categories=cats_gender)

cats_major_discipline = ['STEM', 'Humanities', 'Business Degree', 'Arts', 'Other', 'No Major', 'Unknown']
data.major_discipline = pd.Categorical(data.major_discipline, ordered=True, categories=cats_major_discipline)

cats_company_type = ['Pvt Ltd', 'Public Sector', 'Funded Startup', 'Early Stage Startup', 'NGO', 'Other', 'Unknown']
data.company_type = pd.Categorical(data.company_type, ordered=True, categories=cats_company_type)

In [167]:
#### TRANSFORM EXPERIENCE AND LAST NEW JOB INTO NUMERIC VALUES ####

# Drop rows where experience and last_new_job is unknow
data.drop(data[(data.experience == 'Unknown') | (data.last_new_job == 'Unknown')].index, axis = 0, inplace = True)

replace_dict_values = {
                        '<1': '0',
                        '>20': '21',
                        'never': '0',
                        '>4': '5'
                        }

homogeneise_values(data, replace_dict_values)

# Convert to integer type
data.experience = pd.to_numeric(data.experience, downcast = 'integer')
data.last_new_job = pd.to_numeric(data.last_new_job, downcast='integer')

In [168]:
#### UNIQUE VALUES ####

for col in data. columns:
    describe_col(data, col)
    print('\n--------------------')

ENROLLEE_ID (enrollee_id)

General stats
count     1809.000000
mean     16936.647319
std       9668.703393
min         30.000000
25%       8578.000000
50%      16873.000000
75%      25308.000000
max      33353.000000
Name: enrollee_id, dtype: float64

Value distribution
32403    1
28799    1
31485    1
29417    1
8335     1
        ..
30670    1
27245    1
853      1
5258     1
12215    1
Name: enrollee_id, Length: 1809, dtype: int64

Missing values
0

--------------------
CITY (city)

General stats
count         1809
unique         102
top       city_103
freq           422
Name: city, dtype: object

Value distribution
city_103    422
city_21     254
city_16     149
city_114    133
city_160    100
           ... 
city_107      1
city_155      1
city_139      1
city_31       1
city_84       1
Name: city, Length: 102, dtype: int64

Missing values
0

--------------------
CITY_DEVELOPMENT_INDEX (city_development_index)

General stats
count    1809.000000
mean        0.831786
std         0.

In [169]:
data.to_csv('../data/HR_Data_cleaned.csv')